In [1]:
import geopandas
import r5py
import shapely
import pandas as pd
import geopandas as gpd
import numpy as np
import datetime

import os
import snman
from snman import osmnx_customized as oxc
from snman.constants import *

PERIMETER = '_accessibility_debug'

# Set these paths according to your own setup
data_directory = os.path.join('C:',os.sep,'Users','lballo','polybox','Research','SNMan','SNMan Shared','data_v2')
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)
outputs_path = os.path.join(data_directory, 'outputs', PERIMETER)

CRS_internal = 2056      # for Zurich

C:\Users\lballo\AppData\Roaming\Python\Python39\site-packages\osmnx\utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


In [2]:
# Import Statent grid
statent = gpd.read_parquet(
    os.path.join(inputs_path, 'switzerland', 'switzerland', 'statent', 'STATENT_2021_ebc_10_reduced_fields.gzip')
)
statent.rename(columns={'RELI': 'id'}, inplace=True)
#statent

In [48]:
print('read transport network car')
transport_network_car_before = r5py.TransportNetwork(
    os.path.join(outputs_path, 'tt_car_before.osm.pbf'),
)

transport_network_car_after = r5py.TransportNetwork(
    os.path.join(outputs_path, 'tt_car_after.osm.pbf'),
)

read transport network car


In [51]:
# filter statent for cells within distance limit
from_id, to_id = (68232483, 68232484)

origins = geopandas.GeoDataFrame(
    {
        "id": [from_id],
        #"geometry": [statent.set_index('id').geometry[from_id]]
        "geometry": shapely.Point(2680843, 1249291)
    },
    crs="EPSG:2056"
)

destinations = geopandas.GeoDataFrame(
    {
        "id": [to_id],
        #"geometry": [statent.set_index('id').geometry[to_id]]
        "geometry": shapely.Point(2681956, 1249512)
    },
    crs="EPSG:2056"
)

tt_computer = r5py.DetailedItinerariesComputer(
    transport_network_car_before,
    transport_modes=[r5py.TransportMode.CAR],
    origins=origins,
    destinations=destinations,
    departure=datetime.datetime(2024, 2, 22, 8, 30),
    
)
tt_matrix = tt_computer.compute_travel_details()

for key in ['transport_mode', 'travel_time']:
    tt_matrix[key] = tt_matrix[key].astype(str)

snman.io.export_gdf(
    tt_matrix,
    os.path.join(outputs_path, 'routes.gpkg')
)

AttributeError: 'RegionalTask' object has no attribute 'fromLon'